In [1]:
import getpass
import msal #pip install msal in anaconda
import logging
import requests #pip install requests_negotiate_sspi
import json


class SpecialSession(requests.Session):
    def request(self, method, url, *args, **kwargs):
        if 'oauth2/v2.0/token' in url:
            kwargs['proxies'] = {
                'https': 'http://inet-proxy-a.adns.ubs.net:8085',
                'http': 'http://inet-proxy-a.adns.ubs.net:8085'
            }
        return super().request(method, url, verify=False, *args, **kwargs)

def get_token():
    scopevar = "app://ubscloud.onmicrosoft.com/AT10200/PROD/Panservice/OData.Read.All"
    clietnid = "42f45e42-33c4-4be1-a028-cf0bb6b3b248"

    session = SpecialSession()
    scope = [scopevar]
    # Create a preferably long-lived app instance which maintains a token cache.
    app = msal.PublicClientApplication(
        clietnid,
        authority="https://login.microsoftonline.com/fb6ea403-7cf1-4905-810a-fe5547e98204",
        # token_cache=...  # Default cache is in memory only.
        # You can learn how to use SerializableTokenCache from
        # https://msal-python.readthedocs.io/en/latest/#msal.SerializableTokenCache
        http_client=session
    )
    # The pattern to acquire a token looks like this.
    result = None
    # Firstly, check the cache to see if this end user has signed in before
    accounts = app.get_accounts("rene.brupbacher@ubs.com")
    if accounts:
        logging.info("Account(s) exists in cache, probably with token too. Let's try.")
        print("Account(s) already signed in:")
        for a in accounts:
            print(a["username"])
        chosen = accounts[0]  # Assuming the end user chose this one to proceed
        print("Proceed with account: %s" % chosen["username"])
        # Now let's try to find a token in cache for this account
        result = app.acquire_token(scope, account=chosen)
    if not result:
        logging.info("No suitable token exists in cache. Let's get a new one from AAD.")
        print("A local browser window will be open for you to sign in. CTRL+C to cancel.")
        result = app.acquire_token_interactive(
            scope
        )
    return result

# region get token / print token
token = get_token()
access_token = str(token)

# endregion
my_headers = {'Authorization': "Bearer {}".format(token.get('access_token'))}

url = 'https://saturn-pan.azure.ubs.net/odata'



C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'login.microsoftonline.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'login.microsoftonline.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


A local browser window will be open for you to sign in. CTRL+C to cancel.


C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'inet-proxy-a.adns.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
import json
import asyncio
import random
import requests
from requests_negotiate_sspi import HttpNegotiateAuth
import xml.dom.minidom
import datetime
from datetime import datetime, timedelta

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

import nest_asyncio
nest_asyncio.apply()

# for calculating mean_squared error
from sklearn.metrics import mean_squared_error

import openpyxl as xl


#UAT
#url = 'https://saturn-app.ubstest.net:5002/odata'

#PROD
#url = 'https://saturn-app.ubs.net:5002/odata'




import timeit
#starttime = timeit.default_timer()



In [3]:

Curr = datetime.today()
tday = Curr.strftime("%Y-%m-%d")

date_1yr = (Curr - timedelta(days=365)).strftime("%Y-%m-%d")
date_1m = (Curr - timedelta(days=30)).strftime("%Y-%m-%d")







In [4]:
#Get a/c Id in saturn for Europe '105' & Global '186' & Ichiysohi '192' & Nomura '186'
r = requests.get(url+'/Accounts?$filter=GIM2_ACCOUNT_ID in (%2702093094%27,%2700572794%27,%2702299194%27,%2702237894%27) &$Select=ACCOUNT_ID', verify=False, headers=my_headers)
#r = requests.get(url+'/Accounts', verify=False, headers=my_headers) 
json_content = r.json()

instrument = json.dumps(json_content, indent = 4,sort_keys=False)
instrDump = json.loads(instrument)
account = pd.DataFrame(instrDump["value"])
account

C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


,ACCOUNT_ID
0,103
1,105
2,186
3,192


In [5]:
pos_tday = (Curr - timedelta(days=5)).strftime("%Y-%m-%d")

In [6]:
import datetime

from pandas.tseries.offsets import BDay

today = datetime.datetime.today()
BD = today - BDay(1)
BD = BD.strftime("%Y-%m-%d")


In [7]:


rr = requests.get(url + '/Holdings?$filter=ACCOUNT_ID in (103,105,186,192) and HOLDING_VALUE_DATE Ge ' + BD + ' and HOLDING_VALUE_DATE Le '+ BD +' and SOURCE_SYSTEM_ID eq %27DATAHUB%27 &$Select=ACCOUNT_ID,HOLDING_AMOUNT,INSTRUMENT_ID',  verify=False,headers=my_headers)
#rr = requests.get(url + '/Holdings?$filter=ACCOUNT_ID in (105,186) and HOLDING_VALUE_DATE Ge ''2021-08-17'' and HOLDING_VALUE_DATE Le ''2021-08-17'' and SOURCE_SYSTEM_ID eq %27DATAHUB%27 &$Select=ACCOUNT_ID,HOLDING_VALUE_DATE,HOLDING_AMOUNT,INSTRUMENT_ID',  verify=False, auth=HttpNegotiateAuth()) 
#rr = requests.get(urlParam, verify=False, auth=HttpNegotiateAuth()) 
 

json_content = rr.json()

instrument = json.dumps(json_content, indent = 4,sort_keys=False)
instrDump = json.loads(instrument)
pos = pd.DataFrame(instrDump["value"])
#instrument


C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [8]:

#pos.to_excel("Output_Saturn/positions1.xlsx",index=False,na_rep='NA')


In [9]:
position = pos['INSTRUMENT_ID'].apply(str)

In [12]:
position = pd.DataFrame(position)

In [13]:
position1 = position.iloc[:250]
position2 = position.iloc[250:]

In [14]:
pos1 = position1.INSTRUMENT_ID.str.cat(sep=',')
pos2 = position2.INSTRUMENT_ID.str.cat(sep=',')

In [15]:
#GETTING isin
#rr = requests.get(url + '/Instruments?$filter=instrument_ID in (375740,390512) &$Select=instrument_ID,ISIN', verify=False, auth=HttpNegotiateAuth())
#rr = requests.get(url + '/Instruments?$filter=instrument_ID in (375740,390512) &$Select=instrument_ID,ISIN', verify=False, auth=HttpNegotiateAuth())

#rr = requests.get(url +'/Instruments?$filter=instrument_ID in (375740,390512) &$Select=instrument_ID,ISIN', verify=False, headers=my_headers)
rr = requests.get(url +'/Instruments?$filter=instrument_ID in (' + pos1 + ') &$Select=instrument_ID,ISIN', verify=False, headers=my_headers)

json_content = rr.json()



instrument = json.dumps(json_content, indent = 4,sort_keys=False)
instrDump = json.loads(instrument)
isin1 = pd.DataFrame(instrDump["value"])

C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [16]:
rr = requests.get(url +'/Instruments?$filter=instrument_ID in (' + pos2 + ') &$Select=instrument_ID,ISIN', verify=False, headers=my_headers)

json_content = rr.json()



instrument = json.dumps(json_content, indent = 4,sort_keys=False)
instrDump = json.loads(instrument)
isin2 = pd.DataFrame(instrDump["value"])

C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [17]:
isin = isin1.append(isin2)
isin.drop_duplicates()

,INSTRUMENT_ID,ISIN
0,20015,XS0742395287
1,20074,US530715AG61
2,20168,XS1057356773
3,129541,XS1140296614
4,205844,XS1414094927
...,...,...
78,824182,XS2296021798
80,827053,XS2257580857
94,850921,XS2284144339
116,943525,XS2353011724


In [18]:
pos['HOLDING'] = pos.groupby(['INSTRUMENT_ID'])['HOLDING_AMOUNT'].transform('sum')
pos.drop_duplicates(subset=['INSTRUMENT_ID'], keep = 'first', inplace = True)


In [19]:
#pos.to_excel("Output_Saturn/positions2.xlsx",index=False,na_rep='NA')

In [20]:
Holdings = pd.merge(pos,isin,how='left',on='INSTRUMENT_ID')



In [21]:
#Holdings['HOLDING'] = Holdings.groupby(['INSTRUMENT_ID'])['HOLDING_AMOUNT'].transform('sum')

In [22]:
#Hlds = Holdings.drop_duplicates(subset=['INSTRUMENT_ID'])

In [23]:
#Holdings.to_excel("Output_Saturn/Holdings.xlsx",index=False,na_rep='NA')


In [24]:
def concatenate(dd):
    flds = ''
    for index, row in dd.iterrows():
        if row['on/off']=='on':
            if flds=='':
                flds = str(row['field'])   
            else:
                flds += ',' + str(row['field'])                  
                

    return flds   

In [25]:
xls = pd.ExcelFile('field_selecter.xls')
field_static = pd.read_excel(xls, 'static_PP')
field_dynamic = pd.read_excel(xls, 'dynamic_PP')
field_price = pd.read_excel(xls, 'price')

In [26]:
fldsPRICE = concatenate(field_price)
fldsINSTRTD = concatenate(field_dynamic)


In [27]:
#Instrument (Static Data)

fldsINSTR = concatenate(field_static)


r = requests.get(url+'/Instruments?$filter=INSTRUMENT_TP_IDS eq %27CONVERTIBLE_BOND%27 &$Select='+fldsINSTR, verify=False, headers=my_headers)
#r = requests.get('https://saturn-pan.azure.ubs.net/odata/InstrumentTDs?$filter=VALUE_DATE Ge 2020-05-31 and VALUE_DATE Le 2022-01-21&$orderby=VALUE_DATE desc &$Select=ACCRUED_VALUE,CREDIT_SPREAD,INSTRUMENT_ID,DELTA,GAMMA,IMPLIED_VOLA,INPUT_VOLATILITY,PARITY,PREMIUM,THEORETICAL_PRICE,VALUE_DATE,VEGA', verify=False, headers=my_headers)
json_content = r.json()

instrument = json.dumps(json_content, indent = 4,sort_keys=False)
instrDump = json.loads(instrument)
instr = pd.DataFrame(instrDump["value"])
#instr


C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


In [28]:
#retrieves last update date

import sqlite3 as sql
conn = sql.connect('saturn.db')

cursor = conn.cursor()
cursor.execute("SELECT MAX(VALUE_DATE) AS maximum FROM 'cb'")
result=cursor.fetchall()
value_date= result[0][0]
print(value_date)
conn.close()


2023-04-13


In [29]:
import datetime
import time
from datetime import date
from datetime import datetime, timedelta

end_date = date.today()



In [30]:
MaxDate = datetime.strptime(value_date,'%Y-%m-%d').date()

DaysDelta = (end_date - MaxDate).days -1

In [31]:
start_date = (end_date - timedelta(days=DaysDelta))
start_date

datetime.date(2023, 4, 14)

In [32]:
       
    
resultList = []
resultListPrice = []

async def callRequest(url):

    r = requests.get(url, verify=False, headers=my_headers)   
    
    json_content = r.json()
    json_dump = json.dumps(json_content, indent = 4,sort_keys=False)
    json_load = json.loads(json_dump)
    res = pd.DataFrame(json_load["value"])
    

    return res
   


    
async def main(urlINSTRTD, urlPRICE, DaysDelta):
    
    coroutines = []
    coroutinesPrice = []
    
    #import datetime
    #end_date = datetime.datetime(2020, 1, 1).date()
    #start_date = datetime.datetime(2019, 1, 1).date()
  
    end_date = date.today()
    start_date = (end_date - timedelta(days=DaysDelta))
    delta = timedelta(days=10)
    end_date_tmp = start_date + delta

    while start_date <= end_date:

        end_date_tmp = start_date + delta
        url = 'https://saturn-pan.azure.ubs.net/odata/InstrumentTDs?$filter=VALUE_DATE ge ' + str(start_date) + ' and  VALUE_DATE lt ' + str(end_date_tmp) + '  &$Select=' + fldsINSTRTD

        coroutines.append(callRequest(url))
        start_date += delta
        

    
    #end_date = datetime.datetime(2021, 12, 31).date()
    #start_date = datetime.datetime(2021, 1, 1).date()
  
    end_date = date.today()
    start_date = (end_date - timedelta(days=DaysDelta))
    delta = timedelta(days=3)
    end_date_tmp = start_date + delta 
        
        
    while start_date <= end_date:

        end_date_tmp = start_date + delta
        url = 'https://saturn-pan.azure.ubs.net/odata/Pricings?$filter=PRICE_DATE ge ' + str(start_date) + ' and  PRICE_DATE lt ' + str(end_date_tmp) + '  &$Select=' + fldsPRICE
        coroutinesPrice.append(callRequest(url))
        start_date += delta
        
     
    
    completed, pending = await asyncio.wait(coroutines)
    for i in completed:
        resultList.append(i.result())
        
        
        
        
    completed, pending = await asyncio.wait(coroutinesPrice)
    for j in completed:
        resultListPrice.append(j.result())    

       
        
            

if __name__ == '__main__':
    
    event_loop = asyncio.get_event_loop()
    try:
        event_loop.run_until_complete(main(fldsINSTRTD, fldsPRICE, DaysDelta))
    finally:
        event_loop.close()
        


 

        
        


C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\ProgramData\Miniconda3\envs\jup385\lib\site-packages\urllib3\connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'saturn-pan.azure.ubs.net'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warning

RuntimeError: Cannot close a running event loop

In [33]:
resultListPrice

[      INSTRUMENT_ID                 PRICE_DATE PRICE_CCY  PRICE_CLOSE
 0            720926  2023-04-14T00:00:00+02:00       USD      75.0819
 1            901014  2023-04-14T00:00:00+02:00       USD      73.4743
 2            746588  2023-04-14T00:00:00+02:00       EUR     102.7500
 3            799491  2023-04-14T00:00:00+02:00       EUR      46.5580
 4            488301  2023-04-14T00:00:00+02:00       EUR      98.3470
 ...             ...                        ...       ...          ...
 1894        1018411  2023-04-14T00:00:00+02:00       USD      96.3017
 1895        1020937  2023-04-14T00:00:00+02:00       EUR      45.0280
 1896        1020938  2023-04-14T00:00:00+02:00       AUD      95.0000
 1897        1024040  2023-04-14T00:00:00+02:00       USD     105.1000
 1898        1024367  2023-04-14T00:00:00+02:00       EUR      97.3150
 
 [1899 rows x 4 columns],
 Empty DataFrame
 Columns: []
 Index: []]

In [34]:
resultList

[     INSTRUMENT_ID  ACCRUED_VALUE  CREDIT_SPREAD      DELTA     GAMMA  \
 0           720926       1.043750         1200.0  10.392678  0.002066   
 1           901014       0.000000          600.0  32.951603  0.002999   
 2           746588       0.000000          450.0   1.383205  0.002226   
 3           799491       0.121816          140.0  47.800123  0.005275   
 4           488301       0.029863            5.0   0.034800  0.000078   
 ..             ...            ...            ...        ...       ...   
 937         786393       0.452778          775.0   1.527920  0.001125   
 938         842827       0.000000        10000.0   6.004158  0.001423   
 939         778113       0.129861          525.0  81.217870  0.002692   
 940         709809       0.137500         2000.0  22.420043  0.003456   
 941         980439       0.630208         2000.0  75.513667  0.002456   
 
      IMPLIED_VOLA  INPUT_VOLATILITY      PARITY     PREMIUM  \
 0        0.000000              60.0   18.1131

In [35]:
i = 0
j = len(resultList)

instrTD = pd.DataFrame()


for i in range(j):
    instrTD = instrTD.append(resultList[i])
    
    

In [36]:
i = 0
j = len(resultListPrice)

priceTD = pd.DataFrame()


for i in range(j):
    priceTD = priceTD.append(resultListPrice[i])

In [37]:
#instrTD.to_excel("Output_Saturn/instrTD.xlsx",index=False,na_rep='NA')
#priceTD.to_excel("Output_Saturn/priceTD.xlsx",index=False,na_rep='NA')

In [38]:
idl =instr[['INSTRUMENT_ID','REFERENCE_ID']]

In [39]:
#add reference ID (EQ) to InstrTD. Keeps only active positions which are equities (REFERNCE_ID)
instrTD = instrTD.merge(idl,on='INSTRUMENT_ID',how='inner')




In [40]:
#add merge key for later merge

instrTD['merge_key_eq'] = instrTD['REFERENCE_ID'].astype(str) + instrTD['VALUE_DATE']
instrTD['merge_key_cb'] = instrTD['INSTRUMENT_ID'].astype(str) + instrTD['VALUE_DATE']

In [41]:
# create merge key id for priceTD
priceTD['merge_key_prices'] = priceTD['INSTRUMENT_ID'].astype(str) + priceTD['PRICE_DATE']

In [42]:
# Merge CB prices with instrTD 

saturnData_cb = pd.merge(instrTD,priceTD,left_on='merge_key_cb',right_on='merge_key_prices',how='inner')
saturnData_cb = saturnData_cb.rename(columns={'PRICE_CLOSE':'PRICE_CB'})
saturnData_cb = saturnData_cb.rename(columns={'INSTRUMENT_ID_x':'INSTRUMENT_ID'})

del saturnData_cb['INSTRUMENT_ID_y']
del saturnData_cb['merge_key_prices']
del saturnData_cb['merge_key_cb']
del saturnData_cb['PRICE_DATE']





In [43]:
# Merge EQ prices with instrTD 

saturnData_1 = pd.merge(saturnData_cb,priceTD,how='inner',left_on='merge_key_eq',right_on='merge_key_prices')
saturnData_1 = saturnData_1.rename(columns={'PRICE_CLOSE':'PRICE_EQ'})
saturnData_1 = saturnData_1.rename(columns={'INSTRUMENT_ID_x':'INSTRUMENT_ID'})
saturnData_1 = saturnData_1.rename(columns={'PRICE_CCY_x':'PRICE_CCY'})

del saturnData_1['INSTRUMENT_ID_y']
del saturnData_1['PRICE_CCY_y']

del saturnData_1['merge_key_eq']
del saturnData_1['merge_key_prices']
del saturnData_1['PRICE_DATE']


In [44]:
# Merge Dynamic (saturnData_1) and static Data(instr)
saturn = pd.merge(saturnData_1,instr,how='outer',on='INSTRUMENT_ID')

saturn = saturn.rename(columns={'REFERENCE_ID_x':'REFERENCE_ID'})
del saturn['REFERENCE_ID_y']

In [45]:

# Convert String to Date
#saturn_all['VALUE_DATE'] = pd.to_datetime(saturn_all['VALUE_DATE'])
#saturn_all['ISSUE_DATE'] = pd.to_datetime(saturn_all['ISSUE_DATE'])

#saturn['ISSUE_DATE'] = pd.to_datetime(saturn['ISSUE_DATE'])
#saturn['MATURITY_DATE'] = pd.to_datetime(saturn['MATURITY_DATE'])
#saturn['NEXT_PUT_DATE'] = pd.to_datetime(saturn['NEXT_PUT_DATE'], errors='coerce') #removes invalid date of 9999
#saturn['NEXT_CALL_DATE'] = pd.to_datetime(saturn['NEXT_CALL_DATE'],errors='coerce') #removes invalid date of 9999



saturn['IMPLIED_VOLA'] = saturn['IMPLIED_VOLA'].replace(0,np.nan)
saturn['IMPLIED_VOLA'] = saturn['IMPLIED_VOLA'].replace('',np.nan)



saturn.sort_values(by=['VALUE_DATE'],inplace=True,ascending=False)

#rawData = saturn_all






In [46]:
#Converting string to value

# start stop and step variables 
start, stop, step = 0, 10, 1

# Removing string from date

saturn['MATURITY_DATE']= saturn['MATURITY_DATE'].str.slice(start, stop, step) 
saturn['ISSUE_DATE']= saturn['ISSUE_DATE'].str.slice(start, stop, step)
saturn['VALUE_DATE']= saturn['VALUE_DATE'].str.slice(start, stop, step) 

# convert the 'Date' column to datetime format

#rawData['MATURITY_DATE'] = rawData['MATURITY_DATE'].astype('datetime64[ns]')
saturn["MATURITY_DATE"] = pd.to_datetime(saturn["MATURITY_DATE"]).dt.date
saturn["ISSUE_DATE"] = pd.to_datetime(saturn["ISSUE_DATE"]).dt.date
saturn["VALUE_DATE"] = pd.to_datetime(saturn["VALUE_DATE"]).dt.date

In [47]:
#cLEAN prices for units
saturn["PRICE_CB_CLEAN"] = np.where(saturn['IS_NOMINAL']=='N',(saturn['PRICE_CB'] - saturn['ACCRUED_VALUE']), saturn['PRICE_CB'])

#Converts Unit prices in %
#rawData['PRICE_%'] = np.where(rawData['IS_NOMINAL']=='N', (rawData['PRICE_CB'] / rawData['DENOMINATION'] * 100) , rawData['PRICE_CB'])

In [48]:
#sorted(saturn)


In [49]:
#insert retrieved data into table
import sqlite3 as sql
conn = sql.connect('saturn.db')
saturn.to_sql('cb', conn, if_exists='append', index=False)

conn.close()




In [50]:
DateStrg = "'" + str((Curr - timedelta(days=450)).date()) + "'"
DateStrgSQL = "Select * FROM cb WHERE VALUE_DATE>" + DateStrg
DateStrgSQL


"Select * FROM cb WHERE VALUE_DATE>'2022-01-22'"

In [51]:
# retrieve data from SQL for the last 2 years
import sqlite3 as sql
conn = sql.connect('saturn.db')
#saturnDB = pd.read_sql("SELECT * FROM cb WHERE VALUE_DATE>'2022-01-01'", conn)
saturnDB = pd.read_sql(DateStrgSQL, conn)

conn.close()

In [52]:
# Delete certain dates

#conn = sql.connect('saturn.db')
#cur = conn.cursor()
#cur.execute("DELETE FROM cb WHERE VALUE_DATE>'2022-02-01'")
#conn.commit()
#conn.close()



In [53]:
saturnDB['VALUE_DATE'].min()

'2022-01-24'

In [54]:
#saturnDB.to_excel("Output_Saturn/DB.xlsx",index=False,na_rep='NA')

In [55]:
saturn = saturnDB

In [56]:
saturn = saturn.sort_values(by=['VALUE_DATE'], ascending=False)

In [57]:
#saturnDB[['VALUE_DATE','DELTA']].to_excel("Output_Saturn/date.xlsx",index=False,na_rep='NA')

In [58]:
import math

def price_regression(instrument):
    
    #results = ()
    #results = pd.DataFrame(columns=['a','b'])
    
    x = instrument.PARITY
    y = instrument.PRICE_CB_CLEAN
    
    z = np.polyfit(x, y, deg=2) #coeficcient
    p = np.poly1d(z) #poly
    c = np.polyval(z,x) #slope
    
    #R2
    yhat = p(x)                      # or [p(z) for z in x]
    ybar = np.sum(y)/len(y)          # or sum(y)/len(y)
    ssreg = np.sum((yhat-ybar)**2)   # or sum([ (yihat - ybar)**2 for yihat in yhat])
    sstot = np.sum((y - ybar)**2)    # or sum([ (yi - ybar)**2 for yi in y])
    R2 = ssreg / sstot
    
    #RMSE Root Mean Squared Error
    RMSE = np.sqrt(mean_squared_error(y,c))
    
    
    #residual standard error  
    #import math
    #y_true = np.array(y)
    #y_predicted = np.array(c)
    #RSS = np.sum(np.square(y_true - y_predicted))
    #rse = math.sqrt(RSS / (len(y_true) - 2))
    #rse = math.sqrt(RSS / (len(y_true)-1))
                    
    return c[0], R2, RMSE, z[0]

In [59]:
saturn['VALUE_DATE'] = pd.to_datetime(saturn['VALUE_DATE'],format='%Y-%m-%d')
saturn['VALUE_DATE'] = saturn['VALUE_DATE'].dt.date


In [60]:
saturn['MATURITY_DATE'] = pd.to_datetime(saturn['MATURITY_DATE'],format='%Y-%m-%d')
saturn['MATURITY_DATE'] = saturn['MATURITY_DATE'].dt.date

In [61]:
oneYr = (Curr - timedelta(days=365))
saturn_1y = saturn[saturn['VALUE_DATE'] > oneYr.date()]

In [62]:
saturn_1y['VALUE_DATE'].min()

datetime.date(2022, 4, 18)

In [63]:
SingleCB = saturn_1y[saturn_1y['INSTR_LONG_NAME'] == 'Cerence 3% 2025']
SingleCB.to_excel("Output_Saturn/SingelCB.xlsx",index=False,na_rep='NA')

In [64]:
#An R2 of 1 indicates that the regression predictions perfectly fit the data.

Reg = saturn_1y.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
#Reg[['slope', 'R2']] = Reg.apply(g, axis=1)

<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-64-4e3d09df7e05>:3: RankWarning: Polyfit may be poorly conditioned
  Reg = saturn_1y.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-64-4e3d09df7e05>:3: RankWarning: Polyfit may be poorly conditioned
  Reg = saturn_1y.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-64-4e3d09df7e05>:3: RankWarning: Polyfit may be poorly conditioned
  Reg = saturn_1y.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-64-4e3d09df7e05>:3: RankWarning: Polyfit may be poorly conditioned
  Reg = saturn_1y.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-64-4e3d09df7e05>:3: RankWarning: Polyfit may be poorly conditioned
  Reg = saturn_1y.groupby('INSTRUM

<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-64-4e3d09df7e05>:3: RankWarning: Polyfit may be poorly conditioned
  Reg = saturn_1y.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-inp

In [65]:
Reg = Reg.to_frame('test')
Reg = Reg['test'].apply(pd.Series)

In [66]:
Reg

,0,1,2,3
INSTRUMENT_ID,,,,
16810,0.200000,100.169884,2.501334e-16,1.427986e-15
16846,1175.795794,0.227364,3.137663e+01,-1.948461e-03
16922,79.698074,0.615380,2.106578e+00,2.040615e-01
16923,76.910476,0.276348,2.664047e+00,4.234493e-01
16999,110.842737,0.961081,1.623395e+00,7.278176e-04
...,...,...,...,...
1023933,101.329387,0.347729,5.679094e-01,-3.485181e-02
1023934,102.235717,0.323902,1.885719e+00,-2.157767e-01
1024366,96.838533,0.941916,2.716758e-01,-6.402867e-02


In [67]:
oneMthEnd = (Curr - timedelta(days=30))
oneMthStart = (Curr - timedelta(days=395))
#oneMth.date()
saturn_1m = saturn[(saturn['VALUE_DATE'] <= oneMthEnd.date()) & (saturn['VALUE_DATE'] >= oneMthStart.date())]
#saturn_1m = (saturn['VALUE_DATE'] >= oneMthStart.date()) & (saturn['VALUE_DATE'] <= oneMthEnd.date())

In [68]:
#SingleCB_1m = saturn_1m[saturn_1m['INSTR_LONG_NAME'] == 'Cerence 3% 2025']
#SingleCB_1m.to_excel("Output_Saturn/SingelCB_1m.xlsx",index=False,na_rep='NA')

In [69]:
Reg_1m = saturn_1m.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))

<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-69-ff0030a904ef>:1: RankWarning: Polyfit may be poorly conditioned
  Reg_1m = saturn_1m.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-69-ff0030a904ef>:1: RankWarning: Polyfit may be poorly conditioned
  Reg_1m = saturn_1m.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-69-ff0030a904ef>:1: RankWarning: Polyfit may be poorly conditioned
  Reg_1m = saturn_1m.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-69-ff0030a904ef>:1: RankWarning: Polyfit may be poorly conditioned
  Reg_1m = saturn_1m.groupby('INSTRUMENT_ID').apply(lambda x: price_regression(x))
<ipython-input-69-ff0030a904ef>:1: RankWarning: Polyfit may be poorly conditioned
  Reg_1m = saturn_1m.g

<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot
<ipython-input-58-2813672645cb>:20: RuntimeWarning: divide by zero encountered in double_scalars
  R2 = ssreg / sstot


In [70]:
Reg_1m = Reg_1m.to_frame('test')
Reg_1m = Reg_1m['test'].apply(pd.Series)

In [71]:
Reg_1m.rename(columns={0: "BINOMINAL_VALUE_1m", 1: 'R2_Score_1m', 2: 'RMSE_1m', 3: 'z_1m'}, inplace=True)

In [72]:
#Reg.rename(columns={0: "BINOMINAL_VALUE", 1: 'R2_Score', 2:'RSE' }, inplace=True)
Reg.rename(columns={0: "BINOMINAL_VALUE", 1: 'R2_Score', 2: 'RMSE', 3: 'z'}, inplace=True)


In [73]:
cols = ['BINOMINAL_VALUE', 'R2_Score','RMSE']
cols_1m = ['BINOMINAL_VALUE_1m', 'R2_Score_1m','RMSE_1m']
Reg[cols] = Reg[cols].round(2)
Reg_1m[cols_1m] = Reg_1m[cols_1m].round(2)

In [74]:
#Reg_1m

In [75]:
#Reg.to_excel("Output_Saturn/Reg.xlsx",index=False,na_rep='NA')

In [76]:
lastDate = saturn_1y['VALUE_DATE'].max()
recentData = saturn_1y[saturn_1y['VALUE_DATE']==lastDate]
#recentData.count()

In [77]:
saturnDataFinal = pd.merge(recentData,Reg,how='left',on='INSTRUMENT_ID')

In [78]:
lastDate_1m = saturn_1m['VALUE_DATE'].max()
recentData_1m = saturn_1m[saturn_1m['VALUE_DATE']==lastDate_1m]

In [79]:
recentData_1m = pd.merge(recentData_1m,Reg_1m,how='left',on='INSTRUMENT_ID')

In [80]:
#Calculating EPSILON 1 month ago
recentData_1m['BINOMINAL_EPSILON_1m'] = (recentData_1m['BINOMINAL_VALUE_1m'] / recentData_1m['PRICE_CB_CLEAN'] - 1) 

In [81]:
#mergin Holdings

#saturnDataFinal = pd.merge(saturnDataFinal,pos,how='left',on='INSTRUMENT_ID')

In [82]:
#Calculating Alpoha
saturnDataFinal['BINOMINAL_EPSILON'] = (saturnDataFinal['BINOMINAL_VALUE'] / saturnDataFinal['PRICE_CB_CLEAN'] - 1) 

In [83]:
#saturnDataFinal['BINOMINAL_EPSILON_ZSCORE'] = ((saturnDataFinal['BINOMINAL_VALUE'] - saturnDataFinal['PRICE_CB_CLEAN']) / saturnDataFinal['RSE']) 

In [84]:
saturnDataFinal = saturnDataFinal.sort_values(by='BINOMINAL_EPSILON', ascending=False)

In [85]:
#Merge with EPSILON 1m
saturnDataFinal = pd.merge(saturnDataFinal,recentData_1m[['INSTRUMENT_ID','BINOMINAL_VALUE_1m','R2_Score_1m','BINOMINAL_EPSILON_1m','z_1m']],how='left',on='INSTRUMENT_ID')



In [86]:
#Merge with HOlding Data
saturnDataFinal = pd.merge(saturnDataFinal,Holdings[['ISIN','HOLDING']],how='left',on='ISIN')


In [87]:
saturnDataFinal.drop_duplicates(inplace = True)

In [88]:
saturnDataFinal['BINOMINAL_EPSILON_CHANGE_ABS'] = (saturnDataFinal['BINOMINAL_EPSILON'] - saturnDataFinal['BINOMINAL_EPSILON_1m'] ) 

In [89]:
saturnDataFinal['VALUE_DATE_1M_END'] = oneMthEnd.date()
saturnDataFinal['VALUE_DATE_1M_START'] = oneMthStart.date()


In [90]:
saturnDataFinal.to_excel("Output_Saturn/EPSILON.xlsx",index=False,na_rep='NA')
saturnDataFinal.to_excel("Q:/DAA/Convertible/Research/Polynomial Model/Python/Universe/EPSILON.xlsx",index=False,na_rep='NA')




In [91]:
tday = Curr
matur = (Curr - timedelta(days=-365))


In [92]:
SaturnEPSILONList_Upside = saturnDataFinal.loc[(saturnDataFinal['BINOMINAL_EPSILON']>0.04) & (saturnDataFinal['ISSUE_SIZE_OUT']>149000000) & (saturnDataFinal['R2_Score']>0.7) & (saturnDataFinal['HOLDING'].isnull()) & (saturnDataFinal['z']>-0.1) & (saturnDataFinal['CREDIT_SPREAD']<800) & (saturnDataFinal['MATURITY_DATE']>matur.date())]

In [93]:
#SaturnEPSILONList_Downside = saturnDataFinal.loc[(saturnDataFinal['BINOMINAL_EPSILON']<-0.06) & (saturnDataFinal['HOLDING']>0) & (saturnDataFinal['R2_Score']>0.7) & (saturnDataFinal['MATURITY_DATE']>matur.date())]
SaturnEPSILONList_Downside = saturnDataFinal.loc[(saturnDataFinal['BINOMINAL_EPSILON']<-0.01) & (saturnDataFinal['HOLDING']>0) & (saturnDataFinal['R2_Score']>0.5) & (saturnDataFinal['PARITY']>50.0)]

In [94]:
SaturnEPSILONList_Upside.to_excel("Q:/DAA/Convertible/Research/Polynomial Model/Python/Top/EPSILONShortlist_Upside.xlsx",index=False,na_rep='NA')
SaturnEPSILONList_Upside.to_excel("Output_Saturn//Regression/Upside/EPSILONShortlist_Upside.xlsx",index=False,na_rep='NA')

In [95]:
SaturnEPSILONList_Downside.to_excel("Q:/DAA/Convertible/Research/Polynomial Model/Python/Flop/EPSILONShortlist_Downside.xlsx",index=False,na_rep='NA')
SaturnEPSILONList_Downside.to_excel("Output_Saturn//Regression/Downside/EPSILONShortlist_Downside.xlsx",index=False,na_rep='NA')


In [96]:
SaturnConcave = saturnDataFinal.loc[(saturnDataFinal['CREDIT_SPREAD']<750) & (saturnDataFinal['PRICE_CB_CLEAN']<140) & (saturnDataFinal['PRICE_CB_CLEAN']<140) & (saturnDataFinal['z']<0) & (saturnDataFinal['R2_Score']>0.7) & (saturnDataFinal['MATURITY_DATE']>matur.date())]
SaturnConcave = SaturnConcave.sort_values(by='z', ascending=True)

In [97]:
SaturnConcave.to_excel("Q:/DAA/Convertible/Research/Polynomial Model/Python/Concave/Concave.xlsx",index=False,na_rep='NA')

In [ ]:
#######END#########